In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from mm_NN import *
import os

cwd = os.getcwd()
pd.set_option('display.expand_frame_repr', False, 'display.max_columns', None)

direc = './stiff_results/'
fname = 'static_LHS_SG_SR_ng'

dataDF = pd.read_csv(direc + fname +'.csv')
input_profs = np.load(direc + fname + '_inputs.npy', allow_pickle=True)

print(dataDF.head())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
   Unnamed: 0       tau     tau_F      tau_SG      tau_SR        m0   x0   a0      a_max         n  resolution   type  t_prime  t1max       x_c       a_c       m_c        dt       ptime  mem_stiff  prime_stiff    mem_time  prime_time
0           0  0.743858  1.487717  528.375005  278.855753  4.074178  0.2  0.2  39.433618  4.836024         0.5  stiff      0.0    0.0  0.916890  1.065067  1.661773  0.500071  208.217373   3.636884    15.227721    0.500071  208.029656
1           1  1.042435  2.084869  212.193700  177.502378  7.308520  0.2  0.2  43.531944  3.196355         0.5  stiff      0.0    0.0  0.816647  1.086740  0.527684  0.500082  474.372130   2.706279     6.296953  225.037042  474.078035
2           2  0.795999  1.591999  706.711448  503.692032  7.988656  0.2  0.2  98.490928  4.558613         0.5  stiff      0.0    0.0  0.906801  1.050135  1.306791  0.500118  137.667567   7.396257    15.742482 

In [ ]:
### figure 3 3D landscapes

# stiff genes
res = 500
m0 = 1.
tau = 1.
n = 4

x_c = x_crit(n)
a_c = alpha_crit(n, tau)
m_c = m_crit_over_m0(n, tau)

params = {}

choose_m = 0.9
# have been using 0.4 and 0.6
params['x_c'] = x_c; params['a_c'] = a_c; params['type'] = 'stiff'; params['m0'] = m0; 
params['tau'] = tau; params['n'] = n
m_c = scipy.optimize.fsolve(m_crit_general, 0.5, args=(params), xtol=1e-10)[0] / params['m0']
# choose_mc_ind = np.where(np.abs(m_space - choose_mc) == np.amin(np.abs(m_space-choose_mc)))[0]

params['m_c'] = m_c

choose_m = m_c

x_space = np.linspace(0, 1.75, res)
a_space = np.linspace(0.2, 2., res)
# x_stiff = 1-np.exp(-choose_m/m0)

ac_ind = np.where(np.abs(a_space - a_c) == np.amin(np.abs(a_space-a_c)))[0]
# print(ac_ind)

U_data = np.zeros((len(x_space), len(a_space)))

for ai, aa in enumerate(a_space):
    for xi, xx in enumerate(x_space):
        U_data[xi, ai] = U(f_m, choose_m, xx, aa, params)


print(np.shape(np.abs(np.diff(U_data, axis=0))))

ind_below = int(ac_ind - res/10)
ind_above = int(ac_ind + res/10)

m0coords = []
m1coords = []
m2coords = []
for ai, aa in enumerate(a_space):
#     xargs = argrelextrema(np.abs(np.diff(U_data[:,ai])), np.less)[0]
    xargs = find_peaks(-np.abs(np.diff(U_data[:,ai])))[0]
    if aa > a_c:
#         xargs = argrelextrema(np.diff(U_data[:,ai], n=2), np.greater)[0]
#         print(np.diff(U_data[:,ai], n=2))
#         if ai == ind_above:
#             fig3, ax3 = plt.subplots(2,1, figsize=(6,6))
#             # np.diff(U_data[:,ai], n=1)[0]
#             ax3[0].plot(x_space, np.insert(np.diff(U_data[:,ai], n=1),0,[np.diff(U_data[:,ai], n=1)[0]]))
#             ax3[1].plot(x_space, U_data[:,ai])
#             sys.exit()
        x_mins = x_space[xargs]
        if len(x_mins) > 1:
            m1coords.append([np.amin(x_mins), aa, U_data[np.amin(xargs), ai]])
            m2coords.append([np.amax(x_mins),aa, U_data[np.amax(xargs), ai]])
        else:
#             print(x_mins)
            if np.abs(np.amin(x_mins) - m1coords[-1][0]) < np.abs(np.amin(x_mins) - m2coords[-1][0]):
                m1coords.append([np.amin(x_mins), aa, U_data[np.amin(xargs), ai]])
            else:
                m2coords.append([np.amax(x_mins),aa, U_data[np.amax(xargs), ai]])
    else:
#         xargs = argrelextrema(np.abs(np.diff(U_data[:,ai])), np.less)[0]
        x_mins = x_space[xargs]
        m0coords.append([np.amin(x_mins),aa, U_data[np.amin(xargs), ai]])

m0coords = np.array(m0coords)
m1coords = np.array(m1coords)
m2coords = np.array(m2coords)